In [1]:
import numpy as np

from os import listdir
from os.path import isfile, join
import networkx as nx
import networkx.algorithms as alg

import dgl
from dgl.nn import RelGraphConv

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


from dgl.nn import GraphConv, AvgPooling
from dgl.nn.pytorch import Sequential

from dgl.dataloading import GraphDataLoader
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

import pickle

from copy import deepcopy
from utils import *

from tqdm import tqdm

In [2]:
    pickle_name = 'nx_dataset.pickle'
with open(pickle_name, 'rb') as f:
    nx_train_set, nx_val_set, nx_test_set = pickle.load(f)


IndentationError: expected an indented block (Temp/ipykernel_27768/2825711784.py, line 3)

In [ ]:
# e2id = {'title':0, 
#         'link':1,
#         'domain':2,
#         'reversed_title':3,
#         'reversed_link':4,
#        }


train_set = add_time_difference(nx_train_set)
val_set = add_time_difference(nx_val_set)
test_set = add_time_difference(nx_test_set)
print('time done')

train_set = add_domain(train_set)
val_set = add_domain(val_set)
test_set = add_domain(test_set)
print('domain done')

train_set = add_sna(train_set)
val_set = add_sna(val_set)
test_set = add_sna(test_set)
print('sna done')


In [ ]:
np.set_printoptions(precision=2)
def add_node_embedding(dataset):
    fvs = []
    labels = []
    for j in range(len(dataset)):
        no_time_flag = 0
        g = dataset[j][0]
        levels = np.zeros(3)
        snas = np.zeros((len(g.nodes), 5))
        time_diffs = []
        for i in range(len(g.nodes)):
            n = g.nodes[i]
            g.nodes[i]['h'] = torch.tensor([
                0.,0., 0.,  # one hot level encoding
                n['time_diff'], n['no_time'],
                *n['sna']
            ])
            snas[i] = n['sna']
            levels[g.nodes[i]['level']] += 1
            if n['no_time'] != 1:
                time_diffs.append(n['time_diff'])
            else: 
                no_time_flag += 1
        links = 0
        titles = 0
        domain = 0
        for e in g.edges:
            if g.edges[e]['link_type']=='link':
                links +=1
                
            if g.edges[e]['link_type']=='title':
                titles +=1
            if g.edges[e]['link_type']=='domain':
                domain += 0.5
                
        levels /= len(g.nodes)
        time_diff_mean = np.asarray(time_diffs).mean()
        time_diff_std = np.asarray(time_diffs).std()
        no_time_flag /= len(g.nodes)
        
        mean_sna = snas.mean(0)
        
        fv = np.array([*levels, time_diff_mean, time_diff_std, no_time_flag,  *mean_sna, len(g.nodes), len(g.edges), links/(links+titles), domain/len(g.nodes)])
        fvs.append(fv)
        labels.append(dataset[j][1])
    return np.asarray(fvs), np.asarray(labels)

train_fvs, train_labels = add_node_embedding(train_set)
val_fvs, val_labels = add_node_embedding(val_set)
test_fvs, test_labels = add_node_embedding(test_set)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from itertools import product

n_estimators = [1, 5, 10, 20, 50, 100]
max_depths = [1, 2, 5, 10, 20, 50, 100]
criterions = ['gini', 'entropy']
params = list(product(n_estimators, criterions, max_depths))
# print(list(params)
train_accs, val_accs = [], []
splits=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

results = []
for n_estimator, criterion, max_depth in params:
    model_results = []
    for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(train_fvs)), train_labels)):
        split_train_fvs = train_fvs[train_idx]
        split_train_labels = train_labels[train_idx]
        split_val_fvs = train_fvs[val_idx]
        split_val_labels = train_labels[val_idx]
        
        clf = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, criterion=criterion, random_state=42)
        clf.fit(split_train_fvs, split_train_labels)
    
#         train_accs.append((clf.predict(train_fvs) == train_labels).mean())
        model_results.append((clf.predict(split_val_fvs)==split_val_labels).mean())
        print('{:.2f}'.format(100*model_results[-1]), end=' ')
    print('\t', np.asarray(model_results).mean())
    results.append(np.asarray(model_results).mean())
    
#     print(val_accs[-1])


In [ ]:
np.set_printoptions(precision=2)
val_accs = np.asarray(results)
val_accs.argmax()
print(f'CV Val acc {100*val_accs.max()}%')

n_estimator, criterion, max_depth = list(params)[val_accs.argmax()]
clf = RandomForestClassifier(n_estimators=n_estimator, criterion=criterion, max_depth=max_depth, random_state=42)
clf.fit(train_fvs, train_labels)
test_acc = 100*(clf.predict(test_fvs)==test_labels).mean()
val_acc =  100*(clf.predict(val_fvs)==val_labels).mean()
print(f'Val acc {val_acc}%')
print(f'best parameters: n_estimators {n_estimator}, criterion {criterion}, max_depth {max_depth}, test accuracy is {test_acc}%')